In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
headers = {'user-agent': 'Mozilla/5.0 (Macintosh Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36'}

In [ ]:
def find_reporter_ettoday(soup):
    elements=soup.find('div',class_="story").find_all('p')
    for i in elements:
        if '報導' in i.text:
            reporter=i.text.split('／')[0].replace("記者",'').strip()
            break
        if '圖文' in i.text:
            reporter=i.text.split('／')[1].replace("記者",'').strip()
            break
    return reporter
def find_report_time_ettoday(soup):
    return soup.find('time',class_="date")['datetime'].split('T')[0]
def find_text_ettoday(soup):
    data_text=''
    text=soup.find('div',class_="story").find_all('p')[1:]
    for i in text:
        if '▲' not in i.text and '►' not in i.text:
            if '／'  in i.text and '報導'  in i.text:
                pass
            else:
                data_text+=i.text
    return data_text

In [ ]:
data_url=[]
data_title=[]
keywords=['颱風','天氣']#補上

In [ ]:
for k in keywords[0:1]:
    count=1
    url_origin='https://www.ettoday.net/news_search/doSearch.php?keywords='+k+'&kind=5&idx=1&page='
    url=url_origin+str(count) #https://udn.com/search/result/2/keywords
    if count==1:
        resp=requests.get(url,headers=headers)
        soup=BeautifulSoup(resp.text,'html5lib')
        stop_page=int(soup.find('p',class_="info").text.split(' | ')[1].replace('共','').replace('頁','').strip())
    while count<=stop_page:
    #for i in range(2):
        try:
            resp=requests.get(url,headers=headers)
            soup=BeautifulSoup(resp.text,'html5lib')
            items=soup.find_all('div',class_="archive clearfix")
            for item in items:
                print(item.find('div',class_="box_2").a.text)
                print(item.find('div',class_="box_2").a['href'])
                data_title.append(item.find('div',class_="box_2").a.text.replace(u'\u3000',u''))
                data_url.append(item.find('div',class_="box_2").a['href'])
            count+=1
            url=url_origin+str(count)
        except:
            print("can't crawl")
            break

In [ ]:
if len(data_title)==len(data_url):
    data=pd.DataFrame()
    data['title']=data_title
    data['url']=data_url
    data['website']='ETtoday'
    print("make dataframe")

In [ ]:
data_reporter=[]
data_text=[]
data_time=[]


count=0
for i in data['url']:
    try:
        resp=requests.get(i,headers=headers)
        resp.encoding='utf-8'
        items=BeautifulSoup(resp.text,"html5lib")
        #items=soup.find_all("div",class_="inner")
        try:
            x=find_reporter_ettoday(items)
            data_reporter.append(x)
            print(x)
        except:
            data_reporter.append("None")
        try:
            y=find_text_ettoday(items)
            data_text.append(y)
            print(y)
        except:
            data_text.append("None")
        try:
            z=find_report_time_ettoday(items)
            data_time.append(z)
            print(z)
        except:
        
            data_time.append("None")
    except:
        print("fail,全部為None")
        data_text.append("None")
        data_time.append("None")
        data_reporter.append("None")
    count+=1
    print("目前執行次數 %d" % count)
    print("目前共有幾筆文章資料 %d" % len(data_text))
    print("目前共有幾筆時間資料 %d" % len(data_time))

In [ ]:
data['reporter']=data_reporter
data['time']=data_time
data['text']=data_text

In [ ]:
data